In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/lib/python3.7/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV and Perform Basic Data Cleaning¶


In [4]:
df = pd.read_csv("../Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 
                        'koi_prad', 'koi_teq', 'koi_insol', 'koi_tce_plnt_num',
                        'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad', ]]

print(selected_features.shape)

(6991, 13)


In [6]:
target = df["koi_disposition"]
target_names = ["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale data
from sklearn.preprocessing import MinMaxScaler

# define min max scaler
scaler = MinMaxScaler(feature_range=(0, 2))
# transform data
X_scaled = scaler.fit_transform(selected_features)

pd.DataFrame(X_scaled).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.101139,0.062127,0.011626,0.062729,0.001125,0.000027,0.048165,0.000002,0.285714,0.005346,0.422214,1.662592,0.009094
1,0.036676,0.081855,0.019225,0.023346,0.014041,0.000144,0.074923,0.000007,0.000000,0.016503,0.482357,1.691555,0.008438
2,0.002758,0.073656,0.025316,0.032368,0.010473,0.000333,0.178799,0.000163,0.000000,0.111343,0.475104,1.699078,0.007582
3,0.004231,0.075561,0.013908,0.021500,0.000773,0.000026,0.180309,0.000169,0.000000,0.008682,0.509256,1.651683,0.010417
4,0.007236,0.077608,0.015118,0.042974,0.000880,0.000026,0.146552,0.000078,0.285714,0.008527,0.511522,1.669739,0.009594


# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, random_state=42)

# Test 
Just making sure this works

In [9]:
from sklearn.svm import SVC
# Create a support vector machine linear classifer and fit it to the training data
model = SVC(kernel='poly')
model.fit(X_train, y_train)

SVC(kernel='poly')

In [10]:
# Print the model score using the test data
model.score(X_test, y_test)

0.6189931350114416

In [11]:
# Calculate the classification report
from sklearn.metrics import classification_report
preds = model.predict(X_test)
print(classification_report(y_test, preds))

                precision    recall  f1-score   support

     CANDIDATE       0.51      0.15      0.23       411
     CONFIRMED       0.53      0.63      0.57       484
FALSE POSITIVE       0.68      0.84      0.75       853

      accuracy                           0.62      1748
     macro avg       0.57      0.54      0.52      1748
  weighted avg       0.60      0.62      0.58      1748



# Create model pipeline

In [12]:
# Create the SVC Model
from sklearn.svm import SVC 
model_exo = SVC(kernel='linear')
model_exo

SVC(kernel='linear')

In [13]:
# Create the SVC Model
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC 
from sklearn import set_config
set_config(display="diagram")

pipe = make_pipeline(model_exo)
pipe

Pipeline(steps=[('svc', SVC(kernel='linear'))])

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [16]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005],
              'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(pipe, param_grid, verbose=3)

In [17]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] svc__C=1, svc__gamma=0.0001, svc__kernel=linear .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  svc__C=1, svc__gamma=0.0001, svc__kernel=linear, score=0.615, total=   0.4s
[CV] svc__C=1, svc__gamma=0.0001, svc__kernel=linear .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  svc__C=1, svc__gamma=0.0001, svc__kernel=linear, score=0.607, total=   0.4s
[CV] svc__C=1, svc__gamma=0.0001, svc__kernel=linear .................


...

[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=poly, score=0.505, total=   0.4s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=poly ..................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=poly, score=0.506, total=   0.4s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=poly ..................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=poly, score=0.506, total=   0.4s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=rbf ...................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=rbf, score=0.505, total=   0.6s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=rbf ...................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=rbf, score=0.506, total=   0.7s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=rbf ...................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=rbf, score=0.505, total=   0.6s
[CV] svc__C=10, svc__gamma=0.0001, svc__kernel=rbf ...................
[CV]  svc__C=10, svc__gamma=0.0001, svc__kernel=rbf, score=0.506, total=   0.6s
[CV] svc__C

[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=linear, score=0.665, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=linear .................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=linear, score=0.629, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=linear .................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=linear, score=0.642, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=linear .................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=linear, score=0.660, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=poly ...................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=poly, score=0.505, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=poly ...................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=poly, score=0.506, total=   0.4s
[CV] svc__C=10, svc__gamma=0.005, svc__kernel=poly ...................
[CV]  svc__C=10, svc__gamma=0.005, svc__kernel=poly, score=0.505, total=   0.4s
[CV] s

[CV]  svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid, score=0.519, total=   0.5s
[CV] svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid ...............
[CV]  svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid, score=0.520, total=   0.5s
[CV] svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid ...............
[CV]  svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid, score=0.520, total=   0.5s
[CV] svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid ...............
[CV]  svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid, score=0.520, total=   0.5s
[CV] svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid ...............
[CV]  svc__C=50, svc__gamma=0.0005, svc__kernel=sigmoid, score=0.523, total=   0.5s
[CV] svc__C=50, svc__gamma=0.001, svc__kernel=linear .................
[CV]  svc__C=50, svc__gamma=0.001, svc__kernel=linear, score=0.662, total=   0.5s
[CV] svc__C=50, svc__gamma=0.001, svc__kernel=linear .................
[CV]  svc__C=50, svc__gamma=0.001, svc__kernel=linear, score=0.693, tota

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  2.7min finished


GridSearchCV(estimator=Pipeline(steps=[('svc', SVC(kernel='linear'))]),
             param_grid={'svc__C': [1, 5, 10, 50],
                         'svc__gamma': [0.0001, 0.0005, 0.001, 0.005],
                         'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [18]:
# List the best parameters for this dataset
print(f"Best params: {grid.best_params_}")
# List the best score
print(f"Best score: {grid.best_score_}")

Best params: {'svc__C': 50, 'svc__gamma': 0.0001, 'svc__kernel': 'linear'}
Best score: 0.6683200649109657


In [19]:
grid.best_estimator_

Pipeline(steps=[('svc', SVC(C=50, gamma=0.0001, kernel='linear'))])

In [20]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names = ["CANDIDATE", "CONFIRMED", "FALSE POSITIVE"]))

                precision    recall  f1-score   support

     CANDIDATE       0.42      0.36      0.39       411
     CONFIRMED       0.58      0.64      0.61       484
FALSE POSITIVE       0.77      0.78      0.78       853

      accuracy                           0.64      1748
     macro avg       0.59      0.59      0.59      1748
  weighted avg       0.64      0.64      0.64      1748



# Save the Model

In [21]:
# save model using joblib
import joblib
filename = 'travis_tyler.sav'
joblib.dump(grid.best_estimator_, filename)

['travis_tyler.sav']